In [1]:
import os
import sys
import glob
import argparse

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Activation
from keras.layers.normalization import BatchNormalization

from keras.optimizers import SGD, RMSprop


import wandb
from wandb.keras import WandbCallback

run = wandb.init()
config = run.config

Using TensorFlow backend.


W&B Run: https://app.wandb.ai/univai-ss2019/kagfish/runs/peyzqh9m
Call `%%wandb` in the cell containing your training loop to display live results.


In [2]:
config.img_width = 299
config.img_height = 299
config.epochs = 50
config.batch_size = 32
config.num_classes = 8

In [3]:
train_dir = "../train"
test_dir = "../valid"

In [4]:
anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF']

In [5]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [6]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [7]:
nb_train_samples = get_nb_files(train_dir)
nb_classes = len(glob.glob(train_dir + "/*"))
nb_valid_smaples = get_nb_files(test_dir)

In [8]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [20]:
folders = glob.glob('../train/*')
records = []
for folder in folders:
    files = glob.glob(folder+"/*.jpg")
    labels = [e.split('/')[2] for e in files]
    train, valid = train_test_split(range(len(files)), test_size=0.2)
    mask = np.zeros(len(files))
    for j in train:
        mask[j] = 1
    for i, label in enumerate(labels):
        d = dict(label=label, file=files[i], train=mask[i])
        records.append(d)
    #print(records)

In [ ]:
folders = glob.glob('train/*')
records = []
for folder in folders:
    files = glob.glob(folder+"/*.jpg")
    labels = [e.split('/')[1] for e in files]
    train, valid = train_test_split(range(len(files)), test_size=0.2)
    mask = np.zeros(len(files))
    for j in train:
        mask[j] = 1
    for i, label in enumerate(labels):
        d = dict(label=label, file=files[i], train=mask[i])
        records.append(d)
import pandas as pd
df = pd.DataFrame.from_records(records)
dftrain = df[df.train==1][['file', 'label']]
dfvalid = df[df.train==0][['file', 'label']]

dftrain.to_csv("tv_train.csv", index=False, header=True)
dfvalid.to_csv("tv_valid.csv", index=False, header=True)

In [23]:
import json
bb_json = {}
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft', 'NoF']

In [28]:

for c in anno_classes:
    j = json.load(open('../bbox/{}_labels.json'.format(c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

{'img_07917.jpg': {'class': 'rect', 'height': 151.06975503141317, 'width': 383.68430384213445, 'x': 547.1578480789353, 'y': 193.3597426816281}, 'img_07915.jpg': {'class': 'rect', 'height': 143.0000000000005, 'width': 98.00000000000036, 'x': 92.00000000000033, 'y': 495.00000000000176}, 'img_07914.jpg': {'class': 'rect', 'height': 85.00000000000034, 'width': 251.00000000000085, 'x': 334.0000000000012, 'y': 244.00000000000085}, 'img_07912.jpg': {'class': 'rect', 'height': 231.0000000000008, 'width': 199.00000000000068, 'x': 388.00000000000136, 'y': 126.00000000000045}, 'img_07904.jpg': {'class': 'rect', 'height': 158.0000000000005, 'width': 73.00000000000027, 'x': 116.00000000000041, 'y': 495.00000000000176}, 'img_07903.jpg': {'class': 'rect', 'height': 150.0000000000005, 'width': 90.00000000000034, 'x': 638.0000000000023, 'y': 332.0000000000012}, 'img_07892.jpg': {'class': 'rect', 'height': 113.0000000000004, 'width': 323.0000000000012, 'x': 275.00000000000097, 'y': 62.00000000000023}, '

In [ ]:

#print(bb_json.keys())
for r in records:
    tot +=1
    name = r['file'].split('/')[-1]
    if not name in keys:
        count += 1
        #print(r['file'])
        r['bbox'] = None
    else:
        bbox = bb_json[name]
        r['x'] = bbox['x']
        r['y'] = bbox['y']
        r['width'] = bbox['width']
        r['height'] = bbox['height']
        records2.append(r)
print("nobbox", count, tot)
print("rec5", records2[:5])
df = pd.DataFrame.from_records(records2)
dftrain = df[df.train==1][['file', 'label', 'x', 'y', 'width', 'height']]
dfvalid = df[df.train==0][['file', 'label', 'x', 'y', 'width', 'height']]

dftrain.to_csv("tv_train.csv", index=False, header=True)
dfvalid.to_csv("tv_valid.csv", index=False, header=True)

In [8]:
def generators(anno_classes, preprocessing_function, img_width, img_height, batch_size=32, binary=False, shuffle=True,
               train_dir="../train", val_dir="../valid"):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

    # this is the augmentation configuration we will use for validation:
    # only rescaling
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True,
        classes = anno_classes,
        class_mode = 'categorical')

    validation_generator = val_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        classes = anno_classes,
        class_mode = 'categorical')
    return train_generator, validation_generator

In [9]:
train_generator, validation_generator = generators(anno_classes, preprocess_input, config.img_width, config.img_height, config.batch_size)


Found 2931 images belonging to 8 classes.
Found 741 images belonging to 8 classes.


In [10]:
conv_base = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))



Instructions for updating:
Colocations handled automatically by placer.


In [11]:
for layer in conv_base.layers:
    layer.trainable = False

In [15]:
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(config.num_classes, activation="softmax"))


KeyError: 'num_classes'

In [ ]:
print(model.summary())

In [17]:

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-3),
              metrics=['acc'])

In [29]:
conv_base = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))


Instructions for updating:
Colocations handled automatically by placer.


In [34]:
for layer in conv_base.layers:
    print(layer.name)

input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_normalization_23
activat

In [37]:
for layer in conv_base.layers[:277]:
    print(layer.name)

input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_normalization_23
activat